In [1]:
%matplotlib inline
%timeit

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import time
import pickle
import datetime
import re
import pyActigraphy

import sys
sys.path.append('/home/ngrav/project/')
from wearables.scripts import utils as wearutils
from wearables.scripts import data as weardata
from wearables.scripts import train as weartrain
from wearables.scripts import eval_ as weareval
from wearables.scripts import model as wearmodels
from wearables.scripts import DTW as weardtw

import torch
import torch.nn as nn
import torch.nn.functional as F

plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=1
plt.rcParams['savefig.dpi'] = 600
sns.set_style("ticks")

In [2]:
pfp = '/home/ngrav/project/wearables/results/'
savefigs = True

In [3]:
# load md info
from wearables.scripts.md_specification import mdpred_voi
from wearables.scripts.aesthetics import md_node_annotation

# filepaths to bst or pre-processed md with calculated metrics 
pp_md_fp = os.path.join(pfp, 'md_v522_220124.csv')

# load up to date md
md = pd.read_csv(pp_md_fp, index_col=0)

threshold = 10
md['Error group'] = 'lt{}wks'.format(threshold)
md.loc[(md['error'] >= threshold), 'Error group'] = 'Higher-than-actual'
md.loc[(md['error'] <= -threshold), 'Error group'] = 'Lower-than-actual'


In [7]:
mfp = '/home/ngrav/project/wearables/model_zoo'

# filepaths to bst or pre-processed md with calculated metrics 
bst_trainer = os.path.join(mfp, 'trainer_itv52_InceptionTime_GA5.pkl')
bst_modelpkl = os.path.join(mfp, '213-itv52_InceptionTime_GA5.pkl')
bst_modelembeds = os.path.join(pfp, 'embeds_v522_220124.csv')

# load up to date md
def loadpkl(fp):
    with open(fp, 'rb') as f:
        return pickle.load(f)
trainer = loadpkl(bst_trainer)

In [11]:
md['gestage_by'].value_counts(dropna=False, normalize=False)

2.0    1229
1.0     710
3.0     436
0.0      49
4.0      39
Name: gestage_by, dtype: int64

In [6]:
md['gestage_by'].value_counts(dropna=False, normalize=True)

2.0    0.498985
1.0    0.288266
3.0    0.177020
0.0    0.019894
4.0    0.015834
Name: gestage_by, dtype: float64

In [10]:
trainer.data.data['cat_class_enc']

{'marital': {'0': 0.0, '1': 1.0},
 'gestage_by': {'-99.0': 0.0, '1.0': 1.0, '2.0': 2.0, '3.0': 3.0, '4.0': 4.0},
 'insur': {'-99.0': 0.0,
  '1.0': 1.0,
  '2.0': 2.0,
  '3.0': 3.0,
  '4.0': 4.0,
  '5.0': 5.0},
 'ethnicity': {'0': 0.0, '1': 1.0},
 'race': {'0': 0.0, '1': 1.0},
 'prior_ptb_all': {'0': 0.0, '1': 1.0},
 'fullterm_births': {'0': 0.0, '1': 1.0},
 'surghx_none': {'0.0': 0.0, '1.0': 1.0},
 'alcohol': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'smoke': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'drugs': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'hypertension': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'pregestational_diabetes': {'0.0': 0.0, '1.0': 1.0, '2.0': 2.0},
 'asthma_yes___1': {'0': 0.0, '1': 1.0},
 'asthma_yes___2': {'0': 0.0, '1': 1.0},
 'asthma_yes___3': {'0': 0.0, '1': 1.0},
 'asthma_yes___4': {'0': 0.0, '1': 1.0},
 'asthma_yes___5': {'0': 0.0, '1': 1.0},
 'asthma_yes___7': {'0': 0.0, '1': 1.0},
 'asthma_yes___8': {'0': 0.0, '1': 1.0},
 'asthma_yes___10': {'0': 0.0, '1': 1.0},
 'as